# **Lab 3 Basim Sherief 1210207**


In [ ]:
# Setup cuda environment
%pip install git+https://github.com/andreinechaev/nvcc4jupyter.git
%load_ext nvcc4jupyter

---


### Previous Requirements

2. It is required to sum all elements in a 3D volume as follows:

- First kernel will sum all elements across the z-dimension and produce a 2D matrix.
- Second kernel will reduce the 2D matrix into a 1D vector.
- The resulting vector should be reduced into a single element using the CPU.
- Print "Kernel {kernel name} Started" after calling each kernel. What do you notice?

---

Example input and output files for the second requirement are attached.

Your code should take two arguments which are the input and output files (you may use C++).


In [ ]:
%%writefile ./cuda_kernels/kernel.cu
#include <stdio.h>
#include <stdlib.h>
#include <cuda_runtime.h>

// Kernel to sum across the z-dimension
__global__ void sumZDimension(double *input, double *output, size_t width, size_t height, size_t depth) {
    size_t x = blockIdx.x * blockDim.x + threadIdx.x;
    size_t y = blockIdx.y * blockDim.y + threadIdx.y;

    if (x < width && y < height) {
        double sum = 0.0;
        for (size_t z = 0; z < depth; ++z) {
            sum += input[(z * height + y) * width + x];
        }
        output[y * width + x] = sum;
    }
}

// Kernel to reduce 2D matrix to 1D vector
__global__ void reduce2DTo1D(double *input, double *output, size_t width, size_t height) {
    size_t x = blockIdx.x * blockDim.x + threadIdx.x;

    if (x < width) {
        double sum = 0.0;
        for (size_t y = 0; y < height; ++y) {
            sum += input[y * width + x];
        }
        output[x] = sum;
    }
}

// Function to reduce 1D vector to a single element on the CPU
double reduce1DToSingle(double *input, size_t length) {
    double sum = 0.0;
    for (size_t i = 0; i < length; ++i) {
        sum += input[i];
    }
    return sum;
}

// Function to read input data from file
void readInputFile(const char *filename, double **data, size_t *width, size_t *height, size_t *depth) {
    FILE *file = fopen(filename, "r");
    if (!file) {
        fprintf(stderr, "Error opening file %s\n", filename);
        exit(EXIT_FAILURE);
    }

    fscanf(file, "%zu %zu %zu", width, height, depth);

    size_t size = (*width) * (*height) * (*depth);
    *data = (double *)malloc(size * sizeof(double));

    for (size_t i = 0; i < size; ++i) {
        fscanf(file, "%lf", &(*data)[i]);
    }

    fclose(file);
}

// Function to write output data to file
void writeOutputFile(const char *filename, double result) {
    FILE *file = fopen(filename, "w");
    if (!file) {
        fprintf(stderr, "Error opening file %s\n", filename);
        exit(EXIT_FAILURE);
    }

    // Round to 3 decimal places
    result = round(result * 1000.0) / 1000.0;

    fprintf(file, "%.3f\n", result);
    fclose(file);
}

int main(int argc, char *argv[]) {
    if (argc != 3) {
        printf("Usage: %s <inputfile> <outputfile>\n", argv[0]);
        return -1;
    }

    const char *inputFile = argv[1];
    const char *outputFile = argv[2];

    // Load input data from file
    double *h_input;
    size_t width, height, depth;
    readInputFile(inputFile, &h_input, &width, &height, &depth);

    // Allocate memory for input and output
    double *d_input, *d_output2D, *d_output1D;
    cudaMalloc(&d_input, width * height * depth * sizeof(double));
    cudaMalloc(&d_output2D, width * height * sizeof(double));
    cudaMalloc(&d_output1D, width * sizeof(double));

    // Copy input data to device
    cudaMemcpy(d_input, h_input, width * height * depth * sizeof(double), cudaMemcpyHostToDevice);

    // Define block and grid sizes
    dim3 blockSize(16, 16);
    dim3 gridSize((width + blockSize.x - 1) / blockSize.x, (height + blockSize.y - 1) / blockSize.y);

    // Launch first kernel
    printf("Kernel sumZDimension Started\n");
    sumZDimension<<<gridSize, blockSize>>>(d_input, d_output2D, width, height, depth);

    // Launch second kernel
    printf("Kernel reduce2DTo1D Started\n");
    reduce2DTo1D<<<(width + blockSize.x - 1) / blockSize.x, blockSize.x>>>(d_output2D, d_output1D, width, height);

    // Copy result back to host
    double *h_output1D = (double *)malloc(width * sizeof(double));
    cudaMemcpy(h_output1D, d_output1D, width * sizeof(double), cudaMemcpyDeviceToHost);
    
    // Reduce 1D vector to a single element on the CPU
    double result = reduce1DToSingle(h_output1D, width);

    // Save result to output file
    writeOutputFile(outputFile, result);

    // Free memory
    cudaFree(d_input);
    cudaFree(d_output2D);
    cudaFree(d_output1D);
    free(h_input);
    free(h_output1D);

    return 0;
}

In [ ]:
# # Compile the CUDA program
# !nvcc cuda_kernels/kernel.cu -o cuda_kernels/kernel.exe

# # Run the executable (Windows style)
# %cd cuda_kernels
# !kernel.exe ../Input_TestCases/inputfile.txt ../Output_TestCases/outputfile_cpu.txt
# %cd ..
# # Correct Answer for TestCase one 110.689
# # perfomance for cuda is depricacted but can be used in kaggle notebook
# # !nvprof .\cuda_kernels/kernel.exe inputfile.txt outputfile_cpu.txt


In [ ]:
# # For profiling with nvprof (Windows style) the new one  for system-level profiling
# # !nsys profile --stats=true .\kernel0.exe Input_TestCases/inputfile.txt Output_TestCases/outputfile_cpu.txt
# # # , for kernel details using Nsight Compute:
# # !ncu --set full .\kernel0 Input_TestCases/inputfile.txt Output_TestCases/outputfile_cpu.txt
# !ncu --metrics gpu__time_duration .\cuda_kernels\kernel.exe Input_TestCases/inputfile.txt Output_TestCases/outputfile_cpu.txt

---
### **Requriments**

modify the last requirement so that the last 1d vector summation is done using the following ways and show their profiling:


### **Requriment - 1**
- Use only 1 block for your kernel and let the CPU handle the final sum.


In [ ]:
%%writefile ./cuda_kernels/kernel1.cu
#include <stdio.h>
#include <stdlib.h>
#include <cuda_runtime.h>
#include <chrono>

// Kernel to sum across the z-dimension
__global__ void sumZDimension(double *input, double *output2D, size_t width, size_t height, size_t depth) {
    size_t x = blockIdx.x * blockDim.x + threadIdx.x;
    size_t y = blockIdx.y * blockDim.y + threadIdx.y;

    if (x < width && y < height) {
        double sum = 0.0;
        for (size_t z = 0; z < depth; ++z) {
            sum += input[(z * height + y) * width + x];
        }
        output2D[y * width + x] = sum;
    }
}

// Kernel to reduce 2D matrix to 1D vector
__global__ void reduce2DTo1D(double *input, double *output, size_t width, size_t height) {
    size_t x = blockIdx.x * blockDim.x + threadIdx.x;
   
    // Reduce the 2d to 1d of length col since i access it in a column wise manner
    if (x < width) {  
             for (size_t i = x; i < width; i += blockDim.x) {
                   double sum = 0.0;
                for (size_t y = 0; y < height; ++y) {
                    sum += input[y * width + i];
                }
                    output[i] = sum;
             }  
    
    }
    
    __syncthreads();
    
    __shared__ double sharedsum[256];
        
    // Reduce the 1d full length of height of matrix to a 1d full length of block dimension 
    if (x < width) {
        double partial_sum = 0.0;
        for (size_t i = x; i < width; i += blockDim.x) {
            partial_sum  +=  output[i];  
        }
        sharedsum[threadIdx.x] = partial_sum; 
    
    }
      
    __syncthreads();
    
    // Clear the output array
    for (size_t i = x; i < width; i += blockDim.x) {
        output[i] = 0.0;
    }
    
    __syncthreads();  
    
    // Store reduced values back into output
      if (threadIdx.x < blockDim.x) {  
          output[threadIdx.x] = sharedsum[threadIdx.x];  
      }
}


// Function to reduce 1D vector to a single element on the CPU
double reduce1DToSingle(double *input1D, size_t length) {
    double sum = 0.0;
    for (size_t i = 0; i < length; ++i) {
        sum += input1D[i];
    }
    return sum;
}
// Simple timing function using standard C library
double get_time_ms() {
    return (double)clock() * 1000.0 / CLOCKS_PER_SEC;  // Return time in ms
}


// Function to read input data from file
void readInputFile(const char *filename, double **data, size_t *width, size_t *height, size_t *depth) {
    FILE *file = fopen(filename, "r");
    if (!file) {
        fprintf(stderr, "Error opening file %s\n", filename);
        exit(EXIT_FAILURE);
    }

    fscanf(file, "%zu %zu %zu", width, height, depth);

    size_t size = (*width) * (*height) * (*depth);
    *data = (double *)malloc(size * sizeof(double));

    for (size_t i = 0; i < size; ++i) {
        fscanf(file, "%lf", &(*data)[i]);
    }

    fclose(file);
}

// Function to write output data to file
void writeOutputFile(const char *filename, double result) {
    FILE *file = fopen(filename, "w");
    if (!file) {
        fprintf(stderr, "Error opening file %s\n", filename);
        exit(EXIT_FAILURE);
    }

    // Round to 3 decimal places
    result = round(result * 1000.0) / 1000.0;

    fprintf(file, "%.3f\n", result);
    fclose(file);
}

// Function to write output array data to file
void writeOutputArrayToFile(const char *filename, double *data) {
    FILE *file = fopen(filename, "w");
    if (!file) {
        fprintf(stderr, "Error opening file %s\n", filename);
        exit(EXIT_FAILURE);
    }

    // Calculate the length of the array
    size_t length = 0;
    while (data[length] != '\0') {
        length++;
    }

    // Write each element of the array to the file, rounded to 3 decimal places
    for (size_t i = 0; i < length; ++i) {
        double rounded_value = round(data[i] * 1000.0) / 1000.0;
        fprintf(file, "%.3f\n", rounded_value);
    }

    fclose(file);
}

int main(int argc, char *argv[]) {
    if (argc != 3) {
        printf("Usage: %s <inputfile> <outputfile>\n", argv[0]);
        return -1;
    }

    const char *inputFile = argv[1];
    const char *outputFile = argv[2];

    // Load input data from file
    double *h_input;
    size_t width, height, depth;
    readInputFile(inputFile, &h_input, &width, &height, &depth);

    // Allocate memory for input and output
    double *d_input, *d_output2D, *d_output1D;
    cudaMalloc(&d_input, width * height * depth * sizeof(double));
    cudaMalloc(&d_output2D, width * height * sizeof(double));
    cudaMalloc(&d_output1D, width * sizeof(double));

    // Copy input data to device
    cudaMemcpy(d_input, h_input, width * height * depth * sizeof(double), cudaMemcpyHostToDevice);

    // Define block and grid sizes
    dim3 blockSize(16, 16);
    dim3 gridSize((width + blockSize.x - 1) / blockSize.x, (height + blockSize.y - 1) / blockSize.y);

    // Launch first kernel
    printf("Kernel sumZDimension Started\n");
    sumZDimension<<<gridSize, blockSize>>>(d_input, d_output2D, width, height, depth);

    // Launch second kernel
    printf("Kernel reduce2DTo1D Started\n");
    reduce2DTo1D<<<1, 256>>>(d_output2D, d_output1D, width, height);


    
    // Copy result back to host
    double *h_output1D = (double *)malloc(width * sizeof(double));
    cudaMemcpy(h_output1D, d_output1D, width * sizeof(double), cudaMemcpyDeviceToHost);

    // Save result to output file
    writeOutputArrayToFile("../Output_TestCases/1d_kernel1.txt", h_output1D);

    
   // Measure time in nanoseconds using chrono
    auto start = std::chrono::high_resolution_clock::now();
    
    // Reduce 1D vector to a single element on the CPU
    double result = reduce1DToSingle(h_output1D, width);
    
    auto end = std::chrono::high_resolution_clock::now();
    auto duration = std::chrono::duration_cast<std::chrono::nanoseconds>(end - start).count();
    
    printf("Execution time of reduce1DToSingle: %lld ns (%.6f ms)\n", duration, duration / 1e6);

    // Save result to output file
    writeOutputFile(outputFile, result);

    // Free memory
    cudaFree(d_input);
    cudaFree(d_output2D);
    cudaFree(d_output1D);
    free(h_input);
    free(h_output1D);

    return 0;
}

In [ ]:
!nvcc cuda_kernels/kernel1.cu -o cuda_kernels/kernel1.exe
%cd cuda_kernels
!kernel1.exe ../Input_TestCases/t2_50.txt ../Output_TestCases/t2_50_k1_o.txt
%cd ..

- **Performance**: Use only **one block** for your kernel and let the **CPU handle the final sum**. Kernel 1

- **Performance Required**: 
  - Time for the vector kernel: `time_for_vec_kernel`
  - Time for the CPU: `time_for_cpu`
  - Memory copy time: `memcopy_time`


In [ ]:
!nsys profile --sample=none --trace=cuda --force-overwrite=true --stats=true --output=custom_profile ./cuda_kernels/kernel1.exe Input_TestCases/t2_50.txt Output_TestCases/t2_50_k1_o.txt


### **Requriment - 2**
- Use only 1 block for your kernal and let one thread to handle the final sum.


In [ ]:
%%writefile ./cuda_kernels/kernel2.cu
#include <stdio.h>
#include <stdlib.h>
#include <cuda_runtime.h>

// Kernel to sum across the z-dimension
__global__ void sumZDimension(double *input, double *output2D, size_t width, size_t height, size_t depth) {
    size_t x = blockIdx.x * blockDim.x + threadIdx.x;
    size_t y = blockIdx.y * blockDim.y + threadIdx.y;

    if (x < width && y < height) {
        double sum = 0.0;
        for (size_t z = 0; z < depth; ++z) {
            sum += input[(z * height + y) * width + x];
        }
        output2D[y * width + x] = sum;
    }
}

// Kernel to reduce 2D matrix to 1D vector
__global__ void reduce2DTo1D(double *input, double *output, size_t width, size_t height) {
    size_t x = blockIdx.x * blockDim.x + threadIdx.x;
   
    // Reduce the 2d to 1d of length col since i access it in a column wise manner
    if (x < width) {  
             for (size_t i = x; i < width; i += blockDim.x) {
                   double sum = 0.0;
                for (size_t y = 0; y < height; ++y) {
                    sum += input[y * width + i];
                }
                    output[i] = sum;
             }  
    
    }
    
    __syncthreads();
    
    __shared__ double sharedsum[256];
        
    // Reduce the 1d full length of height of matrix to a 1d full length of block dimension 
    if (x < width) {
        double partial_sum = 0.0;
        for (size_t i = x; i < width; i += blockDim.x) {
            partial_sum  +=  output[i];  
        }
        sharedsum[threadIdx.x] = partial_sum; 
    
    }
      
    __syncthreads();
    
    // Clear the output array
    for (size_t i = x; i < width; i += blockDim.x) {
        output[i] = 0.0;
    }
    
    __syncthreads();  
    
    // Store reduced values back into output
      if (threadIdx.x < blockDim.x) {  
          output[threadIdx.x] = sharedsum[threadIdx.x];  
      }
      
      if(threadIdx.x == 0){
          for (size_t i = 1; i < blockDim.x; i++) {
              output[0] += output[i];
          }
          // clear the rest of values
           for (size_t i = 1; i < blockDim.x; i++) {
              output[i] = 0;
          }
      }
}

// Function to read input data from file
void readInputFile(const char *filename, double **data, size_t *width, size_t *height, size_t *depth) {
    FILE *file = fopen(filename, "r");
    if (!file) {
        fprintf(stderr, "Error opening file %s\n", filename);
        exit(EXIT_FAILURE);
    }

    fscanf(file, "%zu %zu %zu", width, height, depth);

    size_t size = (*width) * (*height) * (*depth);
    *data = (double *)malloc(size * sizeof(double));

    for (size_t i = 0; i < size; ++i) {
        fscanf(file, "%lf", &(*data)[i]);
    }

    fclose(file);
}

// Function to write output data to file
void writeOutputFile(const char *filename, double result) {
    FILE *file = fopen(filename, "w");
    if (!file) {
        fprintf(stderr, "Error opening file %s\n", filename);
        exit(EXIT_FAILURE);
    }

    // Round to 3 decimal places
    result = round(result * 1000.0) / 1000.0;

    fprintf(file, "%.3f\n", result);
    fclose(file);
}

// Function to write output array data to file
void writeOutputArrayToFile(const char *filename, double *data) {
    FILE *file = fopen(filename, "w");
    if (!file) {
        fprintf(stderr, "Error opening file %s\n", filename);
        exit(EXIT_FAILURE);
    }

    // Calculate the length of the array
    size_t length = 0;
    while (data[length] != '\0') {
        length++;
    }

    // Write each element of the array to the file, rounded to 3 decimal places
    for (size_t i = 0; i < length; ++i) {
        double rounded_value = round(data[i] * 1000.0) / 1000.0;
        fprintf(file, "%.3f\n", rounded_value);
    }

    fclose(file);
}

int main(int argc, char *argv[]) {
    if (argc != 3) {
        printf("Usage: %s <inputfile> <outputfile>\n", argv[0]);
        return -1;
    }

    const char *inputFile = argv[1];
    const char *outputFile = argv[2];

    // Load input data from file
    double *h_input;
    size_t width, height, depth;
    readInputFile(inputFile, &h_input, &width, &height, &depth);

    // Allocate memory for input and output
    double *d_input, *d_output2D, *d_output1D;
    cudaMalloc(&d_input, width * height * depth * sizeof(double));
    cudaMalloc(&d_output2D, width * height * sizeof(double));
    cudaMalloc(&d_output1D, width * sizeof(double));

    // Copy input data to device
    cudaMemcpy(d_input, h_input, width * height * depth * sizeof(double), cudaMemcpyHostToDevice);

    // Define block and grid sizes
    dim3 blockSize(16, 16);
    dim3 gridSize((width + blockSize.x - 1) / blockSize.x, (height + blockSize.y - 1) / blockSize.y);

    // Launch first kernel
    printf("Kernel sumZDimension Started\n");
    sumZDimension<<<gridSize, blockSize>>>(d_input, d_output2D, width, height, depth);

    // Launch second kernel
    printf("Kernel reduce2DTo1D Started\n");
    reduce2DTo1D<<<1, 256>>>(d_output2D, d_output1D, width, height);


    
    // Copy result back to host
    double *h_output1D = (double *)malloc(width * sizeof(double));
    cudaMemcpy(h_output1D, d_output1D, width * sizeof(double), cudaMemcpyDeviceToHost);

    // Save result to output file
    writeOutputArrayToFile("../Output_TestCases/out_kernel2_1.txt", h_output1D);

    // Reduce 1D vector to a single element on the CPU
    double result = h_output1D[0];

    // Save result to output file
    writeOutputFile(outputFile, result);

    // Free memory
    cudaFree(d_input);
    cudaFree(d_output2D);
    cudaFree(d_output1D);
    free(h_input);
    free(h_output1D);

    return 0;
}

In [ ]:
!nvcc cuda_kernels/kernel2.cu -o cuda_kernels/kernel2.exe
%cd cuda_kernels
!kernel2.exe ../Input_TestCases/t2_50.txt ../Output_TestCases/t2_50_k2_o.txt
%cd ..

In [ ]:
!nsys profile --sample=none --trace=cuda --force-overwrite=true --stats=true --output=custom_profile ./cuda_kernels/kernel2.exe Input_TestCases/t2_50.txt Output_TestCases/t2_50_k2_o.txt


### **Requriment - 3**
- Use multiple blocks for your kernal and let the CPU handle the final sum.


In [ ]:
%%writefile ./cuda_kernels/kernel3.cu
#include <stdio.h>
#include <stdlib.h>
#include <cuda_runtime.h>
#include <chrono>

// Kernel to sum across the z-dimension
__global__ void sumZDimension(double *input, double *output, size_t width, size_t height, size_t depth) {
    size_t x = blockIdx.x * blockDim.x + threadIdx.x;
    size_t y = blockIdx.y * blockDim.y + threadIdx.y;

    if (x < width && y < height) {
        double sum = 0.0;
        for (size_t z = 0; z < depth; ++z) {
            sum += input[(z * height + y) * width + x];
        }
        output[y * width + x] = sum;
    }
}


__global__ void reduce2DTo1D(double *input, double *output, size_t width, size_t height) {
    size_t x = blockIdx.x * blockDim.x + threadIdx.x;
    size_t thread_Idx = threadIdx.x;
    extern __shared__ double sharedSum[];

    // Each thread computes a partial sum for its global column index
    double sum = 0.0;
    if (x < width) {
        for (size_t y = 0; y < height; ++y) {
            sum += input[y * width + x];
        }
    }

    // Store partial sum in shared memory using local thread index
    sharedSum[thread_Idx] = sum;
    __syncthreads();

    // Reduce partial sums within the block
    for (size_t stride = blockDim.x / 2; stride > 0; stride /= 2) {
        if (thread_Idx < stride) {
            sharedSum[thread_Idx] += sharedSum[thread_Idx + stride];
        }
        __syncthreads();
    }

    // The first thread in each block writes the block's sum to global memory
    if (thread_Idx == 0) {
               printf("Block %d: sharedSum[0] = %f\n", blockIdx.x, sharedSum[0]);
      
        output[blockIdx.x] = sharedSum[0];
    }
}

// Function to reduce 1D vector to a single element on the CPU
double reduce1DToSingle(double *input, size_t length) {
    double sum = 0.0;
    for (size_t i = 0; i < length; ++i) {
        sum += input[i];
    }
    return sum;
}

// Function to read input data from file
void readInputFile(const char *filename, double **data, size_t *width, size_t *height, size_t *depth) {
    FILE *file = fopen(filename, "r");
    if (!file) {
        fprintf(stderr, "Error opening file %s\n", filename);
        exit(EXIT_FAILURE);
    }

    fscanf(file, "%zu %zu %zu", width, height, depth);

    size_t size = (*width) * (*height) * (*depth);
    *data = (double *)malloc(size * sizeof(double));

    for (size_t i = 0; i < size; ++i) {
        fscanf(file, "%lf", &(*data)[i]);
    }

    fclose(file);
}

// Function to write output data to file
void writeOutputFile(const char *filename, double result) {
    FILE *file = fopen(filename, "w");
    if (!file) {
        fprintf(stderr, "Error opening file %s\n", filename);
        exit(EXIT_FAILURE);
    }

    // Round to 3 decimal places
    result = round(result * 1000.0) / 1000.0;

    fprintf(file, "%.3f\n", result);
    fclose(file);
}


// Function to write output array data to file
void writeOutputArrayToFile(const char *filename, double *data) {
    FILE *file = fopen(filename, "w");
    if (!file) {
        fprintf(stderr, "Error opening file %s\n", filename);
        exit(EXIT_FAILURE);
    }

    // Calculate the length of the array
    size_t length = 0;
    while (data[length] != '\0') {
        length++;
    }

    // Write each element of the array to the file, rounded to 3 decimal places
    for (size_t i = 0; i < length; ++i) {
        double rounded_value = round(data[i] * 1000.0) / 1000.0;
        fprintf(file, "%.3f\n", rounded_value);
    }

    fclose(file);
}

int main(int argc, char *argv[]) {
    if (argc != 3) {
        printf("Usage: %s <inputfile> <outputfile>\n", argv[0]);
        return -1;
    }

    const char *inputFile = argv[1];
    const char *outputFile = argv[2];

    // Load input data from file
    double *h_input;
    size_t width, height, depth;
    readInputFile(inputFile, &h_input, &width, &height, &depth);

    // Allocate memory for input and output
    double *d_input, *d_output2D, *d_output1D;
    cudaMalloc(&d_input, width * height * depth * sizeof(double));
    cudaMalloc(&d_output2D, width * height * sizeof(double));
    cudaMalloc(&d_output1D, width * sizeof(double));

    // Copy input data to device
    cudaMemcpy(d_input, h_input, width * height * depth * sizeof(double), cudaMemcpyHostToDevice);

    // Define block and grid sizes
    dim3 blockSize(16, 16);
    dim3 gridSize((width + blockSize.x - 1) / blockSize.x, (height + blockSize.y - 1) / blockSize.y);

    // Launch first kernel
    printf("Kernel sumZDimension Started\n");
    sumZDimension<<<gridSize, blockSize>>>(d_input, d_output2D, width, height, depth);

    // Launch second kernel
    printf("Kernel reduce2DTo1D Started\n");
    
    reduce2DTo1D<<<(width + blockSize.x - 1) / blockSize.x, blockSize.x>>>(d_output2D, d_output1D, width, height);

    // Copy result back to host
    double *h_output1D = (double *)malloc(width * sizeof(double));
    cudaMemcpy(h_output1D, d_output1D, width * sizeof(double), cudaMemcpyDeviceToHost);

    
    // Save result to output file
    writeOutputArrayToFile("../Output_TestCases/1d_kernel3.txt", h_output1D);

    // Measure time in nanoseconds using chrono
    auto start = std::chrono::high_resolution_clock::now();
    
    double result = reduce1DToSingle(h_output1D, (width + blockSize.x - 1) / blockSize.x);
    
    auto end = std::chrono::high_resolution_clock::now();
    auto duration = std::chrono::duration_cast<std::chrono::nanoseconds>(end - start).count();
    
    printf("Execution time of reduce1DToSingle: %lld ns (%.6f ms)\n", duration, duration / 1e6);


    // Save result to output file
    writeOutputFile(outputFile, result);

    // Free memory
    cudaFree(d_input);
    cudaFree(d_output2D);
    cudaFree(d_output1D);
    free(h_input);
    free(h_output1D);

    return 0;
}

In [ ]:
!nvcc cuda_kernels/kernel3.cu -o cuda_kernels/kernel3.exe
%cd cuda_kernels
!kernel3.exe ../Input_TestCases/t2_50.txt ../Output_TestCases/t2_50_k3_o.txt
%cd ..

In [ ]:
!nsys profile --sample=none --trace=cuda --force-overwrite=true --stats=true --output=custom_profile ./cuda_kernels/kernel3.exe Input_TestCases/t2_50.txt Output_TestCases/t2_50_k3_o.txt
